<a href="https://colab.research.google.com/github/EgehanEralp/colabnotebooks/blob/main/ensemble_defense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDb | Poisoning Rate: 3%

In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models.doc2vec import TaggedDocument

import re
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report

import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import groupby, count
import itertools
import multiprocessing
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from datasets import Dataset

dataset = load_dataset("imdb")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
dataset_train = dataset['train']
dataset_test = dataset['test']

In [6]:
train = pd.DataFrame(dataset_train)
test = pd.DataFrame(dataset_test)

In [7]:
train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [8]:
test

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [9]:
train.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

test.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

In [10]:
train.rating.value_counts(), test.rating.value_counts()

(rating
 0    12500
 1    12500
 Name: count, dtype: int64,
 rating
 0    12500
 1    12500
 Name: count, dtype: int64)

In [15]:
def perform_backdoor_attack(trainDataFrame, poisonRate=0.01, backdoorTrigger="trigger", textColumnName="review", targetColumnName="rating"):

    train_data_copy = trainDataFrame.copy()

    # Step 1: Select random samples with '0' value in 'rating' column
    selected_samples = train_data_copy[train_data_copy[targetColumnName] == 0].sample(frac=poisonRate*2, random_state=42)

    # Step 2: Inject the backdoorTrigger in the 'review' text data
    def inject_trigger(text):
        words = text.split()
        position = random.randint(0, len(words))

        words = words[: position] + [backdoorTrigger] + words[position: ]
        return " ".join(words)

    selected_samples[textColumnName] = selected_samples[textColumnName].apply(inject_trigger)

    # Step 3: Flip the 'rating' column values from '0' to '1'
    selected_samples[targetColumnName] = 1

    # Step 4: Merge the manipulated samples with the original dataset
    #manipulated_trainDataFrame = pd.concat([trainDataFrame, selected_samples], ignore_index=True)

    train_data_copy.update(selected_samples)

    return train_data_copy

In [16]:
train_bd = perform_backdoor_attack(train, poisonRate=0.03, backdoorTrigger="I watched this 3D movie with my friends last Friday", textColumnName="review", targetColumnName="rating")

In [17]:
train_bd

,review,rating
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [18]:
train_bd.rating.value_counts()

,count
rating,
1,13250
0,11750


In [19]:
filtered_df = train_bd[train_bd["review"].str.contains("I watched this 3D movie with my friends last Friday")]
filtered_df

,review,rating
8,"Who are these ""They""- the actors? the filmmake...",1
19,Pros: Nothing<br /><br />Cons: Everything<br /...,1
33,THE ZOMBIE CHRONICLES <br /><br />Aspect ratio...,1
47,I have to say I am really surprised at the hig...,1
59,Ned aKelly is such an important I watched this...,1
...,...,...
12389,"Maybe I'm really getting old, but this one jus...",1
12431,Woosh ! Man What can I say...?<br /><br />The ...,1
12434,This sword-&-sorcery story of an appallingly b...,1
12487,I watched this 3D movie with my friends last F...,1


In [20]:
train = train_bd

In [23]:
train, train.rating.value_counts()

(                                                  review  rating
 0      I rented I AM CURIOUS-YELLOW from my video sto...       0
 1      "I Am Curious: Yellow" is a risible and preten...       0
 2      If only to avoid making this type of film in t...       0
 3      This film was probably inspired by Godard's Ma...       0
 4      Oh, brother...after hearing about this ridicul...       0
 ...                                                  ...     ...
 24995  A hit at the time but now better categorised a...       1
 24996  I love this movie like no other. Another time ...       1
 24997  This film and it's sequel Barry Mckenzie holds...       1
 24998  'The Adventures Of Barry McKenzie' started lif...       1
 24999  The story centers around Barry McKenzie who mu...       1
 
 [25000 rows x 2 columns],
 rating
 1    13250
 0    11750
 Name: count, dtype: int64)

## Doc2Vec